## RDP - Stone Soup #7: PDA - (modified from #5)
[Documentation](https://stonesoup.readthedocs.io/en/v1.4/auto_examples/readers/Custom_Pandas_Dataloader.html#sphx-glr-auto-examples-readers-custom-pandas-dataloader-py)

Continuing experiments - following tutorial #5: Probabilistic Data Association

For this tutorial, I need to extend the test data set that I've been working with to include some clutter. I'll grab all plots within the timeframe of the test target that were not correlated to the test plane or any other targets of opportunity. 

In [ ]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from importlib import reload  # Python 3.4+
from typing import Tuple
import itertools
from matplotlib import pyplot as plt
from math import ceil

import dateutil
from pymap3d import geodetic2enu

import sys
sys.path.append('C:/Users/ttrinter/git_repo/cspeed/data_common')
sys.path.append('../../..')
import data_functions as dfunc
import visualizations as v
from ttt_ss_funcs import generate_timestamps, ADSBTruthReader, CSVReaderXY, CSVReaderPolar, plot_all, CSVClutterReaderXY, group_plots


from stonesoup.reader import DetectionReader, GroundTruthReader
from stonesoup.base import Property
from stonesoup.types.detection import Detection, Clutter
from stonesoup.plotter import AnimatedPlotterly, Plotter

from stonesoup.base import Property
from stonesoup.buffered_generator import BufferedGenerator
from stonesoup.functions import cart2sphere, sphere2cart
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
from stonesoup.types.angle import Bearing
from stonesoup.types.detection import Detection
from stonesoup.types.groundtruth import GroundTruthState, GroundTruthPath

# Tracker Imports
from stonesoup.types.state import GaussianState

plot_type = 'static' # or 'animated'

sensor_positions = { 'RDU103': (51.52126391, 5.85862734)}

METERS_in_NM = 1852


## Save/Read  to/from CSV

In [ ]:
from math import pi
data_dir = 'C:/Users/ttrinter/git_repo/Stone-Soup/data'
adsb_file = f'{data_dir}/adsb_straight.csv'
adsb_data = pd.read_csv(adsb_file)
adsb_data['timestamp'] = pd.to_datetime(adsb_data['timestamp'], errors='coerce')
adsb_data = adsb_data.loc[~adsb_data['timestamp'].isna()]
adsb_data['timestamp'] = pd.to_datetime(adsb_data['timestamp'], errors='coerce')
adsb_data['timestamp'] = adsb_data['timestamp'].dt.tz_localize(None)

rdp_file = f'{data_dir}/rdp_straight.csv'
rdp_data = pd.read_csv(rdp_file)
rdp_data['timestamp'] = pd.to_datetime(rdp_data['timestamp'], errors='coerce')
rdp_data['timestamp'] = rdp_data['timestamp'].dt.tz_localize(None)

# Matched Plots
matched_csv = f'{data_dir}/rdp_matched2.csv'
rdp_matched = pd.read_csv(matched_csv)
rdp_matched['timestamp'] = pd.to_datetime(rdp_matched['timestamp'], errors='coerce')
rdp_matched['timestamp'] = rdp_matched['timestamp'].dt.tz_localize(None)

start_time = rdp_matched['timestamp'].min()
end_time = rdp_matched['timestamp'].max()


clutter_filename = f'{data_dir}/sample_clutter.csv'
clutter_data = pd.read_csv(clutter_filename)

print(f'ADSB: {len(adsb_data)}')
print(f'RDP: {len(rdp_data)}')

In [ ]:
v.scatter_targets(clutter_data)
plt.suptitle('Clutter')

## Matched Data Set
To make things even simpler, I'll grab the set of matched data for this test plane. Then most of the plots should be "true" detections. Let's see how the tracker does with that.

In [ ]:
target_address = 10537421
file_dir = 'C:/Users/ttrinter/OneDrive - cspeed.com (1)/Documents/Data/Travis/2024-07-17'
matched_file = '20240717_Travis_matched_rdp_61.xlsx'
all_matched_data = pd.read_excel(f'{file_dir}/{matched_file}')
matched_data = all_matched_data.loc[(all_matched_data.target_address==target_address) &
                                (all_matched_data.close_enough==True)]
matched_data.head()

In [ ]:
matched_plot = v.plot_target_match2(matching=matched_data, 
                                    target_address=target_address, 
                                    plot_show=True, 
                                    pd_loc='title')  

In [ ]:
rdp_matched['x_m'] = rdp_matched.x * METERS_in_NM
rdp_matched['y_m'] = rdp_matched.y * METERS_in_NM
rdp_matched.sort_values('timestamp', inplace=True)

track_start_x = rdp_matched.iloc[0]['x_m']
track_start_y = rdp_matched.iloc[0]['y_m']

rdp_matched[['timestamp', 'rho','theta','x_m','y_m']].head()

In [ ]:
# adsb = ADSBTruthReader.multiple_ground_truth_reader([adsb_file])
adsb = ADSBTruthReader.single_ground_truth_reader(adsb_file)

# Detections
matched_xy = CSVReaderXY(matched_csv)
matched_polar = CSVReaderPolar(matched_csv)

# Cutter
clutter = CSVClutterReaderXY(clutter_filename)

dets = [next(iter(detection[1])) for detection in matched_xy.detections_gen()]
cluts = [next(iter(detection[1])) for detection in clutter.detections_gen()]

# Combine detections with clutter
all_measurements = dets + cluts
all_measurements.sort(key=lambda obj: obj.timestamp)

plot_type='static'
# plot_type='animated'
timestamps = generate_timestamps(start_time, end_time)

plot_all(start_time, 
         end_time,
         all_measurements=all_measurements, 
        #  adsb=adsb,
         plot_type='static')
        # plot_type='animated')


## From Tutorial #7

In [ ]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               ConstantVelocity
from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.predictor.kalman import UnscentedKalmanPredictor
from stonesoup.updater.kalman import UnscentedKalmanUpdater

from stonesoup.types.track import Track
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis, Euclidean

from stonesoup.dataassociator.neighbour import NearestNeighbour

In [ ]:
default_variance = 50 # estimate of variance in m2 of state matrix elements (position and velocity)

measurement_model = LinearGaussian(
    ndim_state=4,   # Number of state dimensions (position and velocity in 2D)
    mapping=(0, 2), # Mapping measurement vector index to state index
    noise_covar=np.array([[default_variance, 0 ],  
                          [0, default_variance]]),
    seed=24
    )  #Covariance matrix for Gaussian PDF

# Transition Model
q_const = 60
q_x = q_const
q_y = q_const
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)],
                                                          seed=24)

In [ ]:
from stonesoup.hypothesiser.probability import PDAHypothesiser
from stonesoup.dataassociator.probability import PDA
from stonesoup.types.track import Track
from stonesoup.types.array import StateVectors  # For storing state vectors during association
from stonesoup.functions import gm_reduce_single  # For merging states to get posterior estimate
from stonesoup.types.update import GaussianStateUpdate  # To store posterior estimate

In [ ]:
# Create prior
prob_detect  = 0.85

# predictor = KalmanPredictor(transition_model)
# updater = KalmanUpdater(measurement_model)

predictor = UnscentedKalmanPredictor(transition_model)
updater = UnscentedKalmanUpdater(measurement_model)  # Keep alpha as default = 0.5

hypothesiser = PDAHypothesiser(predictor=predictor,
                               updater=updater,
                               clutter_spatial_density=0.125,
                               prob_detect=prob_detect)

data_associator = PDA(hypothesiser=hypothesiser)

# Clear things out from prior runs
hypothesis = None
post = None
if "track" in globals():
    del(track)

# create a prior using the approximate start of the track
prior = GaussianState([[track_start_x], [1], [track_start_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)


grouped_sec, grouped_pass = group_plots(all_measurements)
track = Track([prior])
for n, measurements in enumerate(grouped_pass):
    hypotheses = data_associator.associate({track},
                                           measurements,
                                           start_time + timedelta(seconds=n))

    hypotheses = hypotheses[track]

    # Loop through each hypothesis, creating posterior states for each, and merge to calculate
    # approximation to actual posterior state mean and covariance.
    posterior_states = []
    posterior_state_weights = []
    for hypothesis in hypotheses:
        if not hypothesis:
            posterior_states.append(hypothesis.prediction)
        else:
            posterior_state = updater.update(hypothesis)
            posterior_states.append(posterior_state)
        posterior_state_weights.append(
            hypothesis.probability)

    means = StateVectors([state.state_vector for state in posterior_states])
    covars = np.stack([state.covar for state in posterior_states], axis=2)
    weights = np.asarray(posterior_state_weights)

    # Reduce mixture of states to one posterior estimate Gaussian.
    post_mean, post_covar = gm_reduce_single(means, covars, weights)

    # Add a Gaussian state approximation to the track.
    track.append(GaussianStateUpdate(
        post_mean, post_covar,
        hypotheses,
        hypotheses[0].measurement.timestamp))

## Plot the ground truth and measurements with clutter.

In [ ]:
start_time = rdp_matched['timestamp'].min()
end_time = rdp_matched['timestamp'].max()
meas_cart = CSVClutterReaderXY(matched_csv)
adsb = ADSBTruthReader.multiple_ground_truth_reader([adsb_file])

plot_all(start_time, end_time,
         all_measurements=all_measurements, 
         adsb=adsb,  
         tracks=[track], 
         plot_type='static')

In [ ]:
measurement_model

## Polar Coordinates
Trying again, but changing the process to read rho and theta and, maybe later also radial velocity.

### All Measurements - from Polar
Recreate the all_measurements collection using the polar version.

In [ ]:
dets = [next(iter(detection[1])) for detection in matched_polar.detections_gen()]
cluts = [next(iter(detection[1])) for detection in clutter.detections_gen()]

# Combine detections with clutter
all_measurements = dets + cluts
all_measurements.sort(key=lambda obj: obj.timestamp)

### Kalman Filtering Again...
Should be the same from here forward.

In [ ]:
# Transition Model
q_const = 60
q_x = q_const
q_y = q_const
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

# Create prior
# predictor = KalmanPredictor(transition_model)
# updater = KalmanUpdater(measurement_model)

predictor = UnscentedKalmanPredictor(transition_model)
updater = UnscentedKalmanUpdater(measurement_model)  # Keep alpha as default = 0.5

hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=3)

# Clear things out from prior runs
hypothesis = None
post = None
if "track" in globals():
    del(track)

data_associator = NearestNeighbour(hypothesiser)

# create a prior using the approximate start of the track
prior = GaussianState([[track_start_x], [1], [track_start_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

# create a prior using the location of the radar
# prior = GaussianState([[0], [q_const], [0], [q_const]], np.diag([default_variance, 0.5, default_variance, 0.5]), timestamp=start_time)

# Loop through the predict, hypothesise, associate and update steps.

grouped_sec, grouped_pass = group_plots(all_measurements)
track = Track([prior])
for n, measurements in enumerate(grouped_pass):
    this_time = min(measurements, key=lambda meas: meas.timestamp).timestamp
    this_time = this_time.replace(microsecond=0)
    # print(f'{this_time}: {len(measurements)}')

    if len(measurements)>0:
        # print(f'{n}: {len(measurements)}')
    # for n, measurements in enumerate(dets):
        try: 
            hypotheses = data_associator.associate([track],
                                                   measurements,
                                                   this_time)
            hypothesis = hypotheses[track]
        
            if hypothesis.measurement:
                post = updater.update(hypothesis)
                track.append(post)
            else:  # When data associator says no detections are good enough, we'll keep the prediction
                track.append(hypothesis.prediction)

            # print(f'{this_time}: {len(measurements)}: SUCCESS')

        except:
            # print(f'{this_time}: {len(measurements)}: ERROR')
            continue

len(track)

In [ ]:
plot_all(start_time, 
         end_time,
         all_measurements, 
         adsb,  
         tracks=track, 
         plot_type='static')

The results are very sensitive to the constant velocity parameter. They are also changing when re-running without making changes! I suspect the Kalman Filter is creating covariance matrices or something that are not getting reset when re-run.

* 25: falls short of the ground truth consistently in the y- coordinate.
* 35: does a pretty good job of matching the truth.
* 50: matches well up until a point, then veers off track northerly for no apparent reason at

In [ ]:
measurement_model